In [31]:
import pandas as pd

df = pd.read_excel('./data/fichier forte croissance 2024 avec urls.xlsx')
df.head(3)

,Rang,Nom de l’entreprise,url de l’entreprise,Taux de croissance : Annuel moyen,Taux de croissance : 2019-2022,Chiffre d’affaires : 2022,Chiffre d’affaires : 2019,Salariés : 2022,Salariés : 2019,Salariés : Créations de postes en 2024*,Total 2024,Secteur,Région
0,1,TapNation,https://www.tap-nation.io/,"303,95 %","6.491,36 %",44.324.280 €,672.460 €,33.0,4,25,NaN,"Divertissement, Culture, Multimédia",Île-de-France
1,2,Bathelec,https://www.bathelec.fr/,"257,23 %","4.458,58 %",14.153.329 €,310.477 €,90.0,8,20,NaN,"Construction, Bâtiment",Île-de-France
2,3,Groupe Courtin,https://courtin-promotion.com/,"229,86 %",3.489 %,47.488.703 €,1.323.172 €,60.0,10,50,NaN,Immobilier,Provence-Alpes-Côte d’Azur-Corse


In [36]:
df.columns = ['Rang', 'Nom_de_l_entreprise', 'url_de_l_entreprise',
       'Taux_de_croissance_Annuel_moyen', 'Taux_de_croissance_2019_2022',
       'Chiffre_d_affaires_2022', 'Chiffre_d_affaires_2019',
       'Salariés_2022', 'Salariés_2019',
       'Salariés_Créations_de_postes_en_2024', 'Total_2024', 'Secteur',
       'Région']

In [40]:
df.head(3)

,Rang,Nom_de_l_entreprise,url_de_l_entreprise,Taux_de_croissance_Annuel_moyen,Taux_de_croissance_2019_2022,Chiffre_d_affaires_2022,Chiffre_d_affaires_2019,Salariés_2022,Salariés_2019,Salariés_Créations_de_postes_en_2024,Total_2024,Secteur,Région
0,1,TapNation,https://www.tap-nation.io/,"303,95 %","6.491,36 %",44.324.280 €,672.460 €,33.0,4,25,NaN,"Divertissement, Culture, Multimédia",Île-de-France
1,2,Bathelec,https://www.bathelec.fr/,"257,23 %","4.458,58 %",14.153.329 €,310.477 €,90.0,8,20,NaN,"Construction, Bâtiment",Île-de-France
2,3,Groupe Courtin,https://courtin-promotion.com/,"229,86 %",3.489 %,47.488.703 €,1.323.172 €,60.0,10,50,NaN,Immobilier,Provence-Alpes-Côte d’Azur-Corse


In [44]:
df.columns, len(df.columns)

(Index(['Rang', 'Nom_de_l_entreprise', 'url_de_l_entreprise',
        'Taux_de_croissance_Annuel_moyen', 'Taux_de_croissance_2019_2022',
        'Chiffre_d_affaires_2022', 'Chiffre_d_affaires_2019', 'Salariés_2022',
        'Salariés_2019', 'Salariés_Créations_de_postes_en_2024', 'Total_2024',
        'Secteur', 'Région'],
       dtype='object'),
 13)

In [48]:
df.to_csv('results/entreprises.csv', index=False)

In [3]:
!docker-compose up -d

time="2024-10-11T11:08:56+01:00" level=warning msg="C:\\Users\\Windows\\Downloads\\Projects\\Hajy8_contact_scraping_fichier_entreprises_forte_croissance_2024\\docker-compose.yaml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion"
 Network hajy8_contact_scraping_fichier_entreprises_forte_croissance_2024_default  Creating
 Network hajy8_contact_scraping_fichier_entreprises_forte_croissance_2024_default  Created
 Container selenium-hub-hajy8_letudiant-project  Creating
 Container selenium-hub-hajy8_letudiant-project  Created
 Container hajy8_contact_scraping_fichier_entreprises_forte_croissance_2024-chrome_1_hajy8_letudiant_project-1  Creating
 Container hajy8_contact_scraping_fichier_entreprises_forte_croissance_2024-chrome_1_hajy8_letudiant_project-1  Created
 Container selenium-hub-hajy8_letudiant-project  Starting
 Container selenium-hub-hajy8_letudiant-project  Started
 Container hajy8_contact_scraping_fichier_entreprises_forte_cr

In [1]:
from ItemScraping import ItemScraping
from ItemStorage import ItemStorage
from Item import Item

from contact_links_classification.ContactLinkModel import ContactLinkModel
from OpenAI_API import ContactOpenAIScraping, FoundersOpenAIClassification, FilterFoundersOneByOneOpenAI
from pageProcessing import PageProcessing
from sentenceProcessing import SentenceProcessing

import pandas as pd
import time
import csv
import codecs
import threading

file_path = 'results/entreprises'

def difference_dic(dic1, dic2):
    # Extraire les valeurs de 'Nom_de_l_entreprise' de dic2 dans un set pour une recherche rapide
    entreprises_dic2 = {d['Nom_de_l_entreprise'] for d in dic2}
    
    # Retourner les dictionnaires dans dic1 qui ne sont pas dans dic2 (selon 'Nom_de_l_entreprise')
    return [d for d in dic1 if d['Nom_de_l_entreprise'] not in entreprises_dic2]
    
def csv_to_dict_list(file_path):
    # Utiliser pandas pour lire le CSV avec un encodage automatique
    df = pd.read_csv(file_path)  # ou 'latin1'
    list_of_dicts = df.to_dict(orient='records')
    # Remplacer \xa0 par un espace vide pour chaque valeur dans le dictionnaire
    for item in list_of_dicts:
        for key, value in item.items():
            if isinstance(value, str):  # Vérifie si la valeur est une chaîne de caractères
                item[key] = value.replace('\xa0', ' ')  # Supprime \xa0
                
    return list_of_dicts
    
# Fonction pour chaque thread qui pop un élément de la liste et traite le dictionnaire
def worker(list_of_dicts, lock_list_of_dicts, results, lock_results, for_task, lock_selenium_grid, lock_print):

    if for_task == 'extract_contact_info' :
        contact_link_classifier = ContactLinkModel()
        contact_link_classifier.load_from_local(model_path='./contact_links_classification/Models/model_0/model_contact_40_maxlen_10_epochs')
        contactOpenAIScraping = ContactOpenAIScraping()
        pageProcessing = PageProcessing()
        sentenceProcessing = SentenceProcessing(max_words_before_phone_number_or_email=30)
        
    if for_task == 'extract_Linkedin_url_from_google' :
        with lock_selenium_grid:
            itemScraping = ItemScraping(with_selenium_grid = True)
        itemScraping.get_google_page()

    if for_task == 'extract_LinkedIn_profiles_of_company_founders' :
        with lock_selenium_grid:
            itemScraping = ItemScraping(with_selenium_grid = False)
        itemScraping.get_linkedin_authentication()
        
    if for_task == 'Founder_Profiles_Classification_with_OpenAi':
        foundersOpenAIClassification = FoundersOpenAIClassification()
        with lock_selenium_grid:
            itemScraping = ItemScraping(foundersOpenAIClassification=foundersOpenAIClassification, with_selenium_grid = False, lock_print=lock_print)
            itemScraping.driver.quit()
            
    if for_task == 'filter_Founder_Profiles_with_OpenAi':
        filterFoundersOneByOneOpenAI = FilterFoundersOneByOneOpenAI()
        with lock_selenium_grid:
            itemScraping = ItemScraping(filterFoundersOneByOneOpenAI=filterFoundersOneByOneOpenAI, with_selenium_grid = False, lock_print=lock_print)
            itemScraping.driver.quit()
            
    while True:
        # Utilisation de 'with lock_list_of_dicts' pour synchroniser l'accès à la liste
        with lock_list_of_dicts:
            if list_of_dicts:
                item_dic = list_of_dicts.pop(0)  # Récupérer un dictionnaire de la liste
            else:
                break  # Si la liste est vide, sortir de la boucle

        # Traiter le dictionnaire (en dehors du verrou pour ne pas bloquer les autres threads)
        item = Item()
        item.init_from_dic(item_dic)
  
        if for_task == 'extract_contact_info' :
            with lock_selenium_grid:
                itemScraping = ItemScraping(url=item.url_de_l_entreprise,  # a modifier
                                            item=item,
                                            contact_link_classifier=contact_link_classifier, contactOpenAIScraping=contactOpenAIScraping,
                                            pageProcessing=pageProcessing, sentenceProcessing=sentenceProcessing)
            result = itemScraping.get_all_contact_page_links()
            
        if for_task == 'extract_Linkedin_url_from_google' :
            result = itemScraping.get_linkedin_url_from_company_name(item)
            if not  result['status']:
                with lock_selenium_grid:
                    itemScraping = ItemScraping()
                itemScraping.get_google_page()

        if for_task == 'extract_LinkedIn_profiles_of_company_founders':
            result = itemScraping.get_LinkedIn_profiles_of_company_founders(item)
            
            if not  result['status']:
                with lock_selenium_grid:
                    itemScraping = ItemScraping(with_selenium_grid = False)
                itemScraping.get_linkedin_authentication()
                
        if for_task == 'Founder_Profiles_Classification_with_OpenAi' :
            result = itemScraping.get_Founder_Profiles_using_OpenAi(item)

        if for_task == 'filter_Founder_Profiles_with_OpenAi' :
            result = itemScraping.filter_Founder_Profiles_using_OpenAi(item)
            
            
        # print(result['data'])

        #Sauvegarder le résultat (en utilisant le verrou pour protéger l'accès à la liste des résultats)
        if result['status']:
            with lock_results:
                ItemStorage(f'{file_path}_with_contact_inf_66', value=result['data'])
                results.append(result['data'])


# Fonction principale pour lancer les threads
def process_with_threads(list_of_dicts_items, num_threads=10, for_task='extract_contact_info'):
    results = []
    # Deux verrous, un pour la liste de dictionnaires et un pour les résultats
    lock_list_of_dicts = threading.Lock()
    lock_results = threading.Lock()
    lock_selenium_grid = threading.Lock()
    lock_print = threading.Lock()
    
    # Créer et lancer les threads
    threads = []
    for _ in range(num_threads):
        t = threading.Thread(target=worker, args=(list_of_dicts_items, lock_list_of_dicts, results, lock_results, for_task, lock_selenium_grid, lock_print))
        t.start()
        threads.append(t)
        time.sleep(1)

    # Attendre que tous les threads aient fini
    for t in threads:
        t.join()

    return results

def main():
    complet_file_path = f'{file_path}.csv'
    complet_file_path = 'results/exceptions.csv'
    complet_file_path = f'{file_path}_with_contact_inf_5.csv'

    list_of_dicts_items = csv_to_dict_list(complet_file_path)
    print(len(list_of_dicts_items))
    
    results = process_with_threads(list_of_dicts_items, num_threads=10, for_task = 'filter_Founder_Profiles_with_OpenAi')
    
    print(f'*'*150)

    ItemStorage(f'{file_path}_with_contact_inf_6', value=results)
    
if __name__ == "__main__":
    
    main()
    # Lire le fichier CSV
    df = pd.read_csv(f'{file_path}_with_contact_inf_6.csv')
    df_sorted = df.sort_values(by='Rang')
    # # Enregistrer dans un fichier Excel
    df_sorted.to_excel(f'{file_path}_with_contact_inf_6.xlsx', index=False, engine='openpyxl')

C:\Users\Windows\OneDrive\Desktop\Projects\Hajy8_contact_scraping_fichier_entreprises_forte_croissance_2024\Item.py:57: SyntaxWarning: invalid escape sequence '\`'
  {self.founder_profile}\nprofiles = {self.profiles}"


3653
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Game Developer @TapNation. Founder, Programmer & Designer @GAMEDEVWORKS : True
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Co-Founder & CEO : True
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Entrepreneur & Investor : True
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

## Work for kspr

In [115]:
import pandas as pd

df_1 = pd.read_excel('kspr_results/kspr_fichier_2.xlsx', engine='openpyxl')
df_2 = pd.read_excel('kspr_results/kspr_second_fichier.xlsx', engine='openpyxl')
df_2

,First Name,Last Name,Job,Picture,Description,Comment,Phones,Phone,Professional Email,Professional Emails,...,Company Type,Company Industries,Company Industry Name,Company Founded On,Company rank For Top Companies,Company Specialities,Company Staff Count,Company Staff Interval,Company Follower Count,List
0,Maxime,Lemarchand,Investisseur,NaN,NaN,NaN,"+33 6 62 41 54 65,+33 6 88 71 85 90",+33 6 62 41 54 65,maxime@nunki.co,maxime@nunki.co,...,Privately Held,Software Development,Software Development,2023.0,NaN,NaN,9.0,11-50,2472.0,Default list
1,Remi,Boisson,COO - Co-Fondateur (nous recrutons !),NaN,NaN,NaN,"+33 7 81 98 49 27,+33 6 32 95 15 23,+33 7 68 1...",+33 7 81 98 49 27,remi.boisson@betclick.com,"remi.boisson@betclick.com,remib@pokerstars.com...",...,Partnership,Staffing and Recruiting,Staffing and Recruiting,2015.0,NaN,"Mise en relation entre professionnels,recrutem...",136.0,51-200,4679.0,Default list
2,Thomas,Landais,Investor,NaN,NaN,NaN,"+33 1 83 35 53 56,+41 79 309 67 20,+33 6 18 03...",+33 1 83 35 53 56,thomas@skello.io,"thomas@skello.io,t.landais@friends-n-family.com",...,Privately Held,Food and Beverage Manufacturing,Food and Beverage Manufacturing,2021.0,NaN,NaN,18.0,11-50,4051.0,Default list
3,Astrid,Menut,Directrice administrative et RH / Responsable ...,NaN,NaN,NaN,+33 6 63 22 48 07,+33 6 63 22 48 07,contact@ecig-arrete.fr,contact@ecig-arrete.fr,...,Privately Held,Retail,Retail,2012.0,NaN,NaN,30.0,51-200,2738.0,Default list
4,Remi,Baert,Président-directeur général,NaN,NaN,NaN,"+33 1 41 16 54 09,+33 1 75 43 29 22,+33 6 63 2...",+33 1 41 16 54 09,r.baert@kumulusvape.fr,"r.baert@kumulusvape.fr,remi@ecig-arrete.fr,rem...",...,Privately Held,Retail,Retail,2012.0,NaN,NaN,30.0,51-200,2738.0,Default list
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,Irina,Semkiv,NaN,NaN,NaN,NaN,NaN,NaN,irina@google.com,irina@google.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Default list
198,Agnãâ¨s,Hamdaoui,NaN,NaN,NaN,NaN,+33 6 79 82 87 53,+33 6 79 82 87 53,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Default list
199,Stephanie,Passeiro,Directrice Administrative Et Financière,NaN,NaN,NaN,NaN,NaN,stephanie.passeiro@ubicast.eu,"stephanie.passeiro@ubicast.eu,spasseiro@satisf...",...,Privately Held,IT Services and IT Consulting,IT Services and IT Consulting,2007.0,NaN,"Video creation and streaming,Lecture Capture,e...",31.0,11-50,2382.0,Default list
200,Jean-Marie,Cognet,Vice Président en charge de l'Enseignement Sup...,NaN,NaN,NaN,"+33 6 43 46 23 38,+33 6 42 93 26 50,+33 6 09 2...",+33 6 43 46 23 38,jean-marie.cognet@ubicast.eu,"jean-marie.cognet@ubicast.eu,jm.cognet@ubicast...",...,Nonprofit,Non-profit Organizations,Non-profit Organizations,2018.0,NaN,NaN,35.0,2-10,23359.0,Default list


In [225]:
first_part = df_2.iloc[153:]
second_part = df_2.iloc[:153]
first_part.to_excel(f'kspr_results/kspr_results_first_part.xlsx', index=False, engine='openpyxl')
second_part.to_excel(f'kspr_results/kspr_results_second_part.xlsx', index=False, engine='openpyxl')
df_2.to_excel(f'kspr_results/kspr_results_all_data.xlsx', index=False, engine='openpyxl')
first_part

,First Name,Last Name,Job,Picture,Description,Comment,Phones,Phone,Professional Email,Professional Emails,...,Company Type,Company Industries,Company Industry Name,Company Founded On,Company rank For Top Companies,Company Specialities,Company Staff Count,Company Staff Interval,Company Follower Count,List
153,Alexandre,Marguerite,"Co-Fondateur, Directeur Général",NaN,NaN,NaN,"+33 6 85 01 52 06,+33 6 69 63 53 37,+33 4 67 7...",+33 6 85 01 52 06,alexandre.marguerite@devensys.com,alexandre.marguerite@devensys.com,...,Privately Held,IT Services and IT Consulting,IT Services and IT Consulting,2013.0,NaN,NaN,58.0,51-200,3469.0,Default list
154,Nicolas,Blasco,Directeur Red Team & Pentest,NaN,NaN,NaN,+33 6 84 06 30 93,+33 6 84 06 30 93,nicolas.blasco@devensys.com,nicolas.blasco@devensys.com,...,Privately Held,IT Services and IT Consulting,IT Services and IT Consulting,2013.0,NaN,NaN,58.0,51-200,3469.0,Default list
155,Michael,Carletto,Chief Sales Officer (CSO),NaN,NaN,NaN,"+33 6 07 65 77 42,+1 732-740-8950,+33 4 67 60 ...",+33 6 07 65 77 42,michael.carletto@merox.io,"michael.carletto@merox.io,michael.carletto@dev...",...,Privately Held,IT Services and IT Consulting,IT Services and IT Consulting,2013.0,NaN,NaN,58.0,51-200,3469.0,Default list
156,Olivier,Couesnon,Directeur du SOC,NaN,NaN,NaN,"+33 6 23 65 58 26,+33 6 38 25 51 48,+33 6 99 2...",+33 6 23 65 58 26,olivier.couesnon@devensys.com,olivier.couesnon@devensys.com,...,Privately Held,IT Services and IT Consulting,IT Services and IT Consulting,2013.0,NaN,NaN,58.0,51-200,3469.0,Default list
157,Leo,Gonzales,Cofondateur,NaN,NaN,NaN,"+33 6 33 42 34 78,+1 240-486-1003,+33 6 30 86 ...",+33 6 33 42 34 78,leo.gonzales@merox.io,leo.gonzales@merox.io,...,Privately Held,IT Services and IT Consulting,IT Services and IT Consulting,2013.0,NaN,NaN,58.0,51-200,3469.0,Default list
158,Jeremy,Wies,Conseiller du Commerce Exterieur de la France,NaN,NaN,NaN,"+33 3 67 10 35 79,+33 9 51 75 17 90,+33 6 87 3...",+33 3 67 10 35 79,jeremy@synovo-group.com,jeremy@synovo-group.com,...,Nonprofit,Civic and Social Organizations,Civic and Social Organizations,1898.0,NaN,NaN,750.0,11-50,32696.0,Default list
159,Michel,Lacombe,Directeur Général,NaN,NaN,NaN,"+33 6 74 13 43 83,+33 7 86 10 74 66,+33 6 98 8...",+33 6 74 13 43 83,michel.lacombe@synovo-group.com,michel.lacombe@synovo-group.com,...,Partnership,Software Development,Software Development,2013.0,NaN,"Développement logiciel,Optimisation logistique",72.0,11-50,1115.0,Default list
160,Victor,Decrion,NaN,NaN,NaN,NaN,"+33 6 72 96 06 74,+33 1 40 74 47 13,+352 27 49...",+33 6 72 96 06 74,vdecrion@lauxera.com,vdecrion@lauxera.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Default list
161,Xavier,Esquiro,Directeur Des Achats,NaN,NaN,NaN,NaN,NaN,xavier.esquiro@be-atex.com,xavier.esquiro@be-atex.com,...,Privately Held,Security and Investigations,Security and Investigations,2008.0,NaN,"ATEX,Détection de gaz fixe,Détection de gaz po...",47.0,11-50,1702.0,Default list
162,Cyrille,Furio,Directeur administratif et financier,NaN,NaN,NaN,+33 6 84 96 98 33,+33 6 84 96 98 33,cyrille.furio@be-atex.com,cyrille.furio@be-atex.com,...,Privately Held,Security and Investigations,Security and Investigations,2008.0,NaN,"ATEX,Détection de gaz fixe,Détection de gaz po...",47.0,11-50,1702.0,Default list


In [215]:
df_3 = pd.read_csv('kspr_results/kspr_results.csv', )
df_3['founder_profile'][53]

"{'profile_url': 'https://www.linkedin.com/in/alexandremarguerite?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABJW8OQBMPP06bibsgqxjQUGJa-EgeFW85I', 'person_name': 'Alexandre MARGUERITE', 'profile_description': 'Co-Fondateur, Directeur Gï¿½nï¿½ral'}"

In [14]:
import pandas as pd

df = pd.read_excel('kspr_results/Fichier cible entreprises en croissance.xlsx', engine='openpyxl')
# df.to_excel(f'kspr_results/Fichier cible entreprises en croissance.xlsx', index=False, engine='openpyxl')
df_subset = df.iloc[214:1062]  # 54:134 # 134:214

# Réorganiser les colonnes pour placer founder_profile_url au début
# cols = ['founder_profile_url'] + [col for col in df_subset.columns if col != 'founder_profile_url']
# df_subset = df_subset[cols]
# df_subset = df_subset[['founder_profile_url']]
# df_subset.to_csv(f'kspr_results/Fichier cible entreprises en croissance partie 54-134.csv', index=False, encoding='utf-8-sig')
df_subset.to_excel(f'kspr_results/Fichier cible entreprises en croissance partie 3 214-1062 for hajy8 08-11-2024.xlsx', index=False, engine='openpyxl')
df_subset

,Rang,Nom_de_l_entreprise,url_de_l_entreprise,phones,emails,addresses,linkedin,founder_profile,profile_description,founder_profile_url,...,Taux_de_croissance_Annuel_moyen,Taux_de_croissance_2019_2022,Chiffre_d_affaires_2022,Chiffre_d_affaires_2019,Salariés_2022,Salariés_2019,Salariés_Créations_de_postes_en_2024,Total_2024,Secteur,Région
214,50,BEAM & SAGALINK,https://beamsagalink.com/,"['+33 1 49 96 54 43', '01 49 96 54 43']","['seeyousoon@beamsagalink.com', 'contact_us@be...","['55 rue de Rivoli, 75001 Paris, France', '55,...",https://fr.linkedin.com/company/beam-sagalink2,{'profile_url': 'https://www.linkedin.com/in/k...,Fondateur Supply Chain Synergy,https://www.linkedin.com/in/khalil-a-a80134b?m...,...,"65,96 %","357,14 %",12.423.258 €,2.717.612 €,65.0,35,20.0,85.0,"Audit, conseil et autres services",Île-de-France
215,245,Ezytail,https://www.ezytail.com/,"['+331 89 400 900', '//+331 89 400 900', '+338...",['contact@ezytail.com'],"['9 rue Jean-Jacques Rousseau, 91350 Grigny, F...",https://fr.linkedin.com/company/ezytail,{'profile_url': 'https://www.linkedin.com/in/s...,Responsable de Partenariats chez Obiz©,https://www.linkedin.com/in/sebastienvaloggia?...,...,"26,8 %","103,89 %",4.967.349 €,2.436.255 €,45.0,9,40.0,85.0,Transport et Logistique,Île-de-France
216,245,Ezytail,https://www.ezytail.com/,"['+331 89 400 900', '//+331 89 400 900', '+338...",['contact@ezytail.com'],"['9 rue Jean-Jacques Rousseau, 91350 Grigny, F...",https://fr.linkedin.com/company/ezytail,{'profile_url': 'https://www.linkedin.com/in/c...,Responsable partenariats : apporter de la visi...,https://www.linkedin.com/in/christophe-biron-6...,...,"26,8 %","103,89 %",4.967.349 €,2.436.255 €,45.0,9,40.0,85.0,Transport et Logistique,Île-de-France
217,49,Obiz,https://obiz-concept.fr/,"['123-456-7890', '123-456-7907', '123-456-7900...",['contact@obiz.fr'],"['456 Elm Street, City, Country', '123 Main St...",https://fr.linkedin.com/company/obiz-concept--...,{'profile_url': 'https://www.linkedin.com/in/j...,Directeur Général 𝐒𝐤𝐢 𝐋𝐨𝐢𝐬𝐢𝐫𝐬 𝐃𝐢𝐟𝐟𝐮𝐬𝐢𝐨𝐧 - 𝐑𝐞𝐝𝐮...,https://www.linkedin.com/in/jessica-protiere?m...,...,"66,39 %","360,66 %",39.990.440 €,8.681.000 €,73.0,30,13.0,86.0,"Publicité, Marketing et Communication",Auvergne-Rhône-Alpes
218,49,Obiz,https://obiz-concept.fr/,"['123-456-7890', '123-456-7907', '123-456-7900...",['contact@obiz.fr'],"['456 Elm Street, City, Country', '123 Main St...",https://fr.linkedin.com/company/obiz-concept--...,{'profile_url': 'https://www.linkedin.com/in/e...,🤝 Fondateur & PDG du Groupe Obiz© S.A 🍀 Eurone...,https://www.linkedin.com/in/enora-manac-h-8a99...,...,"66,39 %","360,66 %",39.990.440 €,8.681.000 €,73.0,30,13.0,86.0,"Publicité, Marketing et Communication",Auvergne-Rhône-Alpes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,299,Groupe SYD,https://www.syd.fr/,"['123-456-7890', '0185733999', '0648529463', '...",[],"['123 Main Street, City, Country', '18 Quai du...",https://fr.linkedin.com/company/groupe-syd,{'profile_url': 'https://www.linkedin.com/in/m...,Associé et Directeur Technique chez SYD Cloud ...,https://www.linkedin.com/in/mclavel?miniProfil...,...,"22,23 %","82,62 %",33.125.417 €,18.139.000 €,381.0,211,100.0,481.0,"Informatique, Digital et High-Tech (installati...",Pays de la Loire
1058,299,Groupe SYD,https://www.syd.fr/,"['123-456-7890', '0185733999', '0648529463', '...",[],"['123 Main Street, City, Country', '18 Quai du...",https://fr.linkedin.com/company/groupe-syd,{'profile_url': 'https://www.linkedin.com/in/e...,Associé et directeur de l'activité SYD Cloud I...,https://www.linkedin.com/in/eric-gugenheim?min...,...,"22,23 %","82,62 %",33.125.417 €,18.139.000 €,381.0,211,100.0,481.0,"Informatique, Digital et High-Tech (installati...",Pays de la Loire
1059,299,Groupe SYD,https://www.syd.fr/,"['123-456-7890', '0185733999', '0648529463', '...",[],"['123 Main Street, City, Country', '18 Quai du...",https://fr.linkedin.com/company/groupe-syd,{'profile_url': 'https://www.linkedin

In [3]:
import pandas as pd

df = pd.read_excel('./kspr_results/Fichier cible entreprises en croissance partie 3 214-1062 for hajy8 08-11-2024.xlsx')
df.head(2)

,Rang,Nom_de_l_entreprise,url_de_l_entreprise,phones,emails,addresses,linkedin,founder_profile,profile_description,founder_profile_url,...,Taux_de_croissance_Annuel_moyen,Taux_de_croissance_2019_2022,Chiffre_d_affaires_2022,Chiffre_d_affaires_2019,Salariés_2022,Salariés_2019,Salariés_Créations_de_postes_en_2024,Total_2024,Secteur,Région
0,50,BEAM & SAGALINK,https://beamsagalink.com/,"['+33 1 49 96 54 43', '01 49 96 54 43']","['seeyousoon@beamsagalink.com', 'contact_us@be...","['55 rue de Rivoli, 75001 Paris, France', '55,...",https://fr.linkedin.com/company/beam-sagalink2,{'profile_url': 'https://www.linkedin.com/in/k...,Fondateur Supply Chain Synergy,https://www.linkedin.com/in/khalil-a-a80134b?m...,...,"65,96 %","357,14 %",12.423.258 €,2.717.612 €,65.0,35,20.0,85.0,"Audit, conseil et autres services",Île-de-France
1,245,Ezytail,https://www.ezytail.com/,"['+331 89 400 900', '//+331 89 400 900', '+338...",['contact@ezytail.com'],"['9 rue Jean-Jacques Rousseau, 91350 Grigny, F...",https://fr.linkedin.com/company/ezytail,{'profile_url': 'https://www.linkedin.com/in/s...,Responsable de Partenariats chez Obiz©,https://www.linkedin.com/in/sebastienvaloggia?...,...,"26,8 %","103,89 %",4.967.349 €,2.436.255 €,45.0,9,40.0,85.0,Transport et Logistique,Île-de-France


In [458]:
df.shape

(848, 21)

In [460]:
import json

def processed_phones(val_phones):
    try:
        if str(val_phones) != 'nan' and str(val_phones).strip() != '':
            def contient_pas_de_lettres(chaine):
                return not any(caractere.isalpha() for caractere in chaine)
                
            phones = json.loads(str(val_phones).replace("'",'"').replace('\\xa0', ' ').replace('//', '').replace('WhatsApp: ', '').replace('Fax: ', ''))
            phones = [ phone.strip() for phone in phones if phone.strip() != '' and contient_pas_de_lettres(phone.strip()) and phone.strip()[:7] not in ['+1-202-','555-123','456-789','+123789','+123456','789-012','+123987','+112233','123-456','123-789','1234567','987-654'] and phone.strip()[:3] not in ['+1 '] and len(phone.strip())>6 ]
            phones = [phone for phone in phones if phone[:3] == '+33' or phone[:2] in ['01','02','03','04','05','07','09','33'] or phone[:4] in ['+ 33','(+33']]
            return list(set(phones))
        else:
            return []
    except Exception as e:
        print('Erroooooooooor : ', e.args[0])
        return val_phones
        
list_phones = list(df['phones'].apply(processed_phones))

In [462]:
len(list([ phone for phones in list_phones for phone in phones]))

1117

In [464]:
df['phones'] = df['phones'].apply(processed_phones)
df['phones'][:10]

0             [01 49 96 54 43, +33 1 49 96 54 43]
1    [+331 89 400 900, +33189400900, +3389400900]
2    [+331 89 400 900, +33189400900, +3389400900]
3             [+33 4 69 55 46 59, 04 69 55 46 59]
4             [+33 4 69 55 46 59, 04 69 55 46 59]
5             [+33 4 69 55 46 59, 04 69 55 46 59]
6             [+33 4 69 55 46 59, 04 69 55 46 59]
7             [+33 4 69 55 46 59, 04 69 55 46 59]
8             [+33 4 69 55 46 59, 04 69 55 46 59]
9             [+33 4 69 55 46 59, 04 69 55 46 59]
Name: phones, dtype: object

In [466]:
df.to_excel(f'kspr_results/Fichier cible entreprises en croissance partie 3 214-1062 for hajy8 08-11-2024 16h00min.xlsx', index=False, engine='openpyxl')

In [468]:
df_without_duplicates = df.drop_duplicates(subset='Nom_de_l_entreprise')
df_without_duplicates.to_excel(f'kspr_results/Fichier cible entreprises en croissance partie 3 214-1062 for hajy8 08-11-2024 16h05min (without company duplicates).xlsx', index=False, engine='openpyxl')

#### add founder_name column

In [57]:
import json
import pandas as pd

df = pd.read_excel('./kspr_results/Fichier cible entreprises en croissance partie 3 214-1062 for hajy8 08-11-2024 16h00min.xlsx')

def process_str_to_json(str):
        str = str.replace('"', "'")  
        str = str.replace("{'", '{"')
        str = str.replace("', '", '", "')
        str = str.replace("\", '", '", "')
        str = str.replace("', \"", '", "')
        str = str.replace("': '", '": "')
        str = str.replace("': \"", '": "')
        str = str.replace("\": '", '": "')
        str = str.replace("'}", '"}')
        str = str.replace("'", "\'")
        str = str.replace("\\'", "\'")
        return str
    
def extract_founder_name(founder_profile):
    founder_profile = process_str_to_json(founder_profile)
    #print(founder_profile)
    founder_name = json.loads(founder_profile)['person_name']
    #print(f'===> {founder_name}')
    return founder_name
    
df['founder_name'] = df['founder_profile'].apply(extract_founder_name)

df.to_excel(f'kspr_results/Fichier cible entreprises en croissance partie 3 214-1062 for hajy8 11-11-2024 11h30min with founder_name column.xlsx', index=False, engine='openpyxl')

{"profile_url": "https://www.linkedin.com/in/khalil-a-a80134b?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAH6EAABNVLonECltNBDrhPAHEIIjIMT4-E", "person_name": "Khalil A.", "profile_description": "Partner & Co-founder"}
===> Khalil A.
{"profile_url": "https://www.linkedin.com/in/sebastienvaloggia?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAA5RKoBS8o4D6lZhSSXFOcYG-3Re5asffk", "person_name": "Sebastien Valoggia", "profile_description": "#Entrepreneur et #Startup lover, expert en #supplychain, #retail, #software passionné par #ecologie #RSE #biodiversité #economie circulaire, heureux d'être passé sous les 4t d'empreinte carbone"}
===> Sebastien Valoggia
{"profile_url": "https://www.linkedin.com/in/christophe-biron-603ab912b?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACAFYyMBddSGdfklBJ-Qp52ILNJeyS0f2VM", "person_name": "christophe biron", "profile_description": "Fondateur Supply Chain Synergy"}
===> christophe biron
{"profile_url": "https://www.linkedin.com/in/jessica-protiere?m

## Work on kspr results

In [43]:
for i in range(10):
    display(df['Rï¿½gion,phones,workEmails,directEmails,enriched'][i].split(',"')[1:3])
    print(f'*'*150)

['+33 6 13 54 69 75"', 'nicolas.doucerain@valumen.com"']

******************************************************************************************************************************************************


['+33 6 43 46 23 38,+33 6 42 93 26 50,+33 6 09 20 06 15"',
 'jean-marie.cognet@ubicast.eu,jm.cognet@ubicast.fr,jm.cognet@ubicast.eu"']

******************************************************************************************************************************************************


['"', 'stephanie.passeiro@ubicast.eu,spasseiro@satisfactory.fr"']

...

******************************************************************************************************************************************************


['+33 1 42 99 10 01,+33 6 18 03 18 22,+33 1 42 99 83 33"',
 'n.zeitoun@progressive-fr.com,n.zeitoun@progressiverecruitment.com,n.zeitoun@computerfutures.fr"']

******************************************************************************************************************************************************


['+33 9 60 02 17 96,+33 2 97 47 39 97"', 'dominique.bodan@solutel.com"']

******************************************************************************************************************************************************


In [45]:
# Extraire les deux valeurs et les ajouter à de nouvelles colonnes
df[['kspr_phones', 'kspr_emails']] = df['Rï¿½gion,phones,workEmails,directEmails,enriched'].apply(
    lambda x: pd.Series(x.split(',"')[1:3])
)

# Afficher le DataFrame pour vérifier les nouvelles colonnes
print(df[['kspr_phones', 'kspr_emails']].head())


                                         kspr_phones  \
0                                 +33 6 13 54 69 75"   
1  +33 6 43 46 23 38,+33 6 42 93 26 50,+33 6 09 2...   
2                                                  "   
3                                 +33 6 79 82 87 53"   
4                                                  "   

                                         kspr_emails  
0                     nicolas.doucerain@valumen.com"  
1  jean-marie.cognet@ubicast.eu,jm.cognet@ubicast...  
2  stephanie.passeiro@ubicast.eu,spasseiro@satisf...  
3                                                  "  
4                                  irina@google.com"  


In [55]:
# Remplacer les caractères ' et " par une chaîne vide dans la colonne kspr_phones
df['kspr_phones'] = df['kspr_phones'].str.replace("'", "").str.replace('"', "")
df['kspr_emails'] = df['kspr_emails'].str.replace("'", "").str.replace('"', "")

# Afficher les résultats pour vérifier les modifications
print(df['kspr_phones'].head())

df.to_excel(f'kspr_results/kspr_results.xlsx', index=False, engine='openpyxl')
df.to_csv(f'kspr_results/kspr_results.csv', index=False, encoding='utf-8-sig')

0                                    +33 6 13 54 69 75
1    +33 6 43 46 23 38,+33 6 42 93 26 50,+33 6 09 2...
2                                                     
3                                    +33 6 79 82 87 53
4                                                     
Name: kspr_phones, dtype: object


## Select True valid_profile_description and sort by column and to_excel

In [3]:
import pandas as pd
file_path = 'results/entreprises'
# Load your DataFrame
df = pd.read_csv(f'{file_path}_with_contact_inf_6.csv')  # Replace "your_file.csv" with your actual file path

# Filter out rows where 'valid_profile_description' is False
df = df[df['valid_profile_description'] == True]

# Drop the 'valid_profile_description' column
df = df.drop(columns=['valid_profile_description'])

df_sorted = df.sort_values(by='Rang')
# # Enregistrer dans un fichier Excel
df_sorted.to_excel(f'{file_path}_with_contact_inf_7.xlsx', index=False, engine='openpyxl')
df_sorted.to_csv(f'{file_path}_with_contact_inf_7.csv', index=False)

In [11]:
import pandas as pd
file_path = 'results/entreprises'
# # Lire le fichier CSV
df = pd.read_csv(f'{file_path}_with_contact_inf_5.csv')
df_sorted = df.sort_values(by='Rang')
# # Enregistrer dans un fichier Excel
df_sorted.to_excel(f'{file_path}_with_contact_inf_5.xlsx', index=False, engine='openpyxl')
df_sorted.to_csv(f'{file_path}_with_contact_inf_5.csv', index=False)

In [1]:
import pandas as pd
df = pd.read_excel(f'results/entreprises_with_contact_inf_1.xlsx')
df.to_csv(f'results/entreprises_with_contact_inf_1.csv', encoding='utf-8-sig', index=False)